In [1]:
!pip install -q Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.9 MB/s eta 0:00:00


# Products Data

In [2]:
import pandas as pd
import numpy as np
from itertools import product
import string
import random

# Random seed for reproducibility
random.seed(0)

In [3]:
# Define unique attribute values
flavors = ['Classic', 'Brown Sugar', 'Matcha', 'Taro', 'Strawberry', 'Passionfruit', 'Mango']
sweetness_levels = ['100%', '75%', '50%', '25%', '0%']
pearl_types = ['Classic', 'Mini', 'Popping', 'Jelly', 'Without']
sizes_oz = [16, 20, 24]

In [4]:
# Generate all possible combinations
all_combinations = list(product(flavors, sweetness_levels, pearl_types, sizes_oz))

# Create a DataFrame for all combinations
df_combinations = pd.DataFrame(all_combinations, columns=['Flavor', 'Sweetness Level', 'Pearl Type', 'Size (oz)'])

print(f"Total possible combinations: {len(df_combinations)}")

Total possible combinations: 525


In [5]:
# Define base price per oz
base_price_per_oz = 0.25

# Define pearl type modifiers
pearl_modifier_dict = {
    'Without': -0.01,
    'Classic': 0.00,
    'Mini': 0.03,
    'Popping': 0.05,
    'Jelly': 0.03
}

# Function to calculate Unit Price ($)
def calculate_unit_price(row):
    modifier = pearl_modifier_dict.get(row['Pearl Type'], 0)
    unit_price = (base_price_per_oz + modifier) * row['Size (oz)']
    return round(unit_price, 2)

# Function to calculate Price per oz ($)
def calculate_price_per_oz(row):
    modifier = pearl_modifier_dict.get(row['Pearl Type'], 0)
    price_per_oz = base_price_per_oz + modifier
    return round(price_per_oz, 2)

# Function to calculate Profit ($)
def calculate_profit(unit_price):
    profit = unit_price * 0.15  # 15% profit margin
    return round(profit, 2)

# Apply the pricing functions to the DataFrame
df_combinations['Unit Price ($)'] = df_combinations.apply(calculate_unit_price, axis=1)
df_combinations['Price per oz ($)'] = df_combinations.apply(calculate_price_per_oz, axis=1)
df_combinations['Profit ($)'] = df_combinations['Unit Price ($)'].apply(calculate_profit)

# Display sample entries
print("Sample Entries after Pricing Calculations:")
display(df_combinations.head())

Sample Entries after Pricing Calculations:


,Flavor,Sweetness Level,Pearl Type,Size (oz),Unit Price ($),Price per oz ($),Profit ($)
0,Classic,100%,Classic,16,4.00,0.25,0.60
1,Classic,100%,Classic,20,5.00,0.25,0.75
2,Classic,100%,Classic,24,6.00,0.25,0.90
3,Classic,100%,Mini,16,4.48,0.28,0.67
4,Classic,100%,Mini,20,5.60,0.28,0.84


In [6]:
# Define abbreviations for attributes
flavor_abbr = {
    'Classic': 'CL',
    'Brown Sugar': 'BS',
    'Matcha': 'MA',
    'Taro': 'TR',
    'Strawberry': 'ST',
    'Passionfruit': 'PF',
    'Mango': 'MN'
}

sweetness_abbr = {
    '100%': '100',
    '75%': '075',
    '50%': '050',
    '25%': '025',
    '0%': '000'
}

pearl_abbr = {
    'Classic': 'C',
    'Mini': 'M',
    'Popping': 'P',
    'Jelly': 'J',
    'Without': 'W'
}

size_abbr = {
    16: '16',
    20: '20',
    24: '24'
}

In [7]:
def generate_product_id(row):
    """
    Generates a unique Product ID based on flavor, sweetness level, pearl type, and size.
    Format: FLV-SSS-P-SZ
    Example: CL-100-C-16
    """
    flavor_code = flavor_abbr.get(row['Flavor'], 'XX')
    sweetness_code = sweetness_abbr.get(row['Sweetness Level'], 'XXX')
    pearl_code = pearl_abbr.get(row['Pearl Type'], 'X')
    size_code = size_abbr.get(row['Size (oz)'], 'XX')
    return f"{flavor_code}-{sweetness_code}-{pearl_code}-{size_code}"

# Apply the Product ID generation function
df_combinations['Product ID'] = df_combinations.apply(generate_product_id, axis=1)

# Display sample Product IDs
print("Sample Product IDs:")
display(df_combinations[['Flavor', 'Sweetness Level', 'Pearl Type', 'Size (oz)', 'Product ID']].head())

Sample Product IDs:


,Flavor,Sweetness Level,Pearl Type,Size (oz),Product ID
0,Classic,100%,Classic,16,CL-100-C-16
1,Classic,100%,Classic,20,CL-100-C-20
2,Classic,100%,Classic,24,CL-100-C-24
3,Classic,100%,Mini,16,CL-100-M-16
4,Classic,100%,Mini,20,CL-100-M-20


In [8]:
print("--- Validation Checks ---\n")

# Check for Duplicate Product IDs
duplicate_ids = df_combinations[df_combinations['Product ID'].duplicated(keep=False)]
num_duplicate_ids = duplicate_ids.shape[0]
print(f"Number of duplicate Product IDs: {num_duplicate_ids}")

if num_duplicate_ids > 0:
    print("Duplicate Product IDs found:")
    display(duplicate_ids)
else:
    print("No duplicate Product IDs found. Dataset is clean.")

# Check for Missing Values
missing_values = df_combinations.isnull().sum()
print("\nMissing Values in Each Column:")
print(missing_values)

if missing_values.sum() > 0:
    print("\nMissing values detected. Please address them accordingly.")
    display(df_combinations[df_combinations.isnull().any(axis=1)])
else:
    print("\nNo missing values found.")

# Check for Valid Price Calculations
# Recalculate Unit Price and Price per oz to verify
df_combinations['Calculated Unit Price ($)'] = df_combinations.apply(calculate_unit_price, axis=1)
df_combinations['Calculated Price per oz ($)'] = df_combinations.apply(calculate_price_per_oz, axis=1)
df_combinations['Calculated Profit ($)'] = df_combinations['Calculated Unit Price ($)'].apply(calculate_profit)

# Check discrepancies
price_mismatch = df_combinations[
    (df_combinations['Unit Price ($)'] != df_combinations['Calculated Unit Price ($)']) |
    (df_combinations['Price per oz ($)'] != df_combinations['Calculated Price per oz ($)']) |
    (df_combinations['Profit ($)'] != df_combinations['Calculated Profit ($)'])
]

num_price_mismatches = price_mismatch.shape[0]
print(f"\nNumber of price calculation mismatches: {num_price_mismatches}")

if num_price_mismatches > 0:
    print("Price calculation mismatches found:")
    display(price_mismatch)
else:
    print("All price calculations are correct.")

# Check for Consistent Size Values
valid_sizes = set(sizes_oz)
invalid_sizes = df_combinations[~df_combinations['Size (oz)'].isin(valid_sizes)]
num_invalid_sizes = invalid_sizes.shape[0]
print(f"\nNumber of invalid Size (oz) entries: {num_invalid_sizes}")

if num_invalid_sizes > 0:
    print("Invalid Size (oz) entries found:")
    display(invalid_sizes)
else:
    print("All Size (oz) entries are valid.")

--- Validation Checks ---

Number of duplicate Product IDs: 0
No duplicate Product IDs found. Dataset is clean.

Missing Values in Each Column:
Flavor              0
Sweetness Level     0
Pearl Type          0
Size (oz)           0
Unit Price ($)      0
Price per oz ($)    0
Profit ($)          0
Product ID          0
dtype: int64

No missing values found.

Number of price calculation mismatches: 0
All price calculations are correct.

Number of invalid Size (oz) entries: 0
All Size (oz) entries are valid.


In [9]:
# Summary of Validations
print("--- Validation Summary ---\n")

# Duplicate Product IDs
print("1. Duplicate Product IDs:")
print(f"Number of duplicate Product IDs: {num_duplicate_ids}")
print("Pass" if num_duplicate_ids == 0 else "Fail", end="\n\n")

# Missing Values
print("2. Missing Values:")
print(f"\nTotal missing values: {missing_values.sum()}")
print("Pass" if missing_values.sum() == 0 else "Fail", end="\n\n")

# Price Calculations
print("3. Price Calculations:")
print(f"\nNumber of price mismatches: {num_price_mismatches}")
print("Pass" if num_price_mismatches == 0 else "Fail", end="\n\n")

# Size (oz) Validation
print("4. Size (oz) Validation:")
print(f"\nNumber of invalid Size (oz) entries: {num_invalid_sizes}")
print("Pass" if num_invalid_sizes == 0 else "Fail", end="\n\n")

print("--- End of Validation ---")

# Display the DataFrame
print("\nCurrent DataFrame:")
display(df_combinations.head())

--- Validation Summary ---

1. Duplicate Product IDs:
Number of duplicate Product IDs: 0
Pass

2. Missing Values:

Total missing values: 0
Pass

3. Price Calculations:

Number of price mismatches: 0
Pass

4. Size (oz) Validation:

Number of invalid Size (oz) entries: 0
Pass

--- End of Validation ---

Current DataFrame:


,Flavor,Sweetness Level,Pearl Type,Size (oz),Unit Price ($),Price per oz ($),Profit ($),Product ID,Calculated Unit Price ($),Calculated Price per oz ($),Calculated Profit ($)
0,Classic,100%,Classic,16,4.00,0.25,0.60,CL-100-C-16,4.00,0.25,0.60
1,Classic,100%,Classic,20,5.00,0.25,0.75,CL-100-C-20,5.00,0.25,0.75
2,Classic,100%,Classic,24,6.00,0.25,0.90,CL-100-C-24,6.00,0.25,0.90
3,Classic,100%,Mini,16,4.48,0.28,0.67,CL-100-M-16,4.48,0.28,0.67
4,Classic,100%,Mini,20,5.60,0.28,0.84,CL-100-M-20,5.60,0.28,0.84


In [10]:
# Remove calculated columns used for validation
df_final = df_combinations.drop(['Calculated Unit Price ($)', 'Calculated Price per oz ($)', 'Calculated Profit ($)'], axis=1)

# Define the desired column order
desired_order = ['Product ID', 'Flavor', 'Sweetness Level', 'Pearl Type', 'Size (oz)',
                'Unit Price ($)', 'Price per oz ($)', 'Profit ($)']

# Reorder the DataFrame columns
df_final = df_combinations[desired_order]

# Display the final DataFrame
print("Final DataFrame:")
display(df_final.head())

Final DataFrame:


,Product ID,Flavor,Sweetness Level,Pearl Type,Size (oz),Unit Price ($),Price per oz ($),Profit ($)
0,CL-100-C-16,Classic,100%,Classic,16,4.00,0.25,0.60
1,CL-100-C-20,Classic,100%,Classic,20,5.00,0.25,0.75
2,CL-100-C-24,Classic,100%,Classic,24,6.00,0.25,0.90
3,CL-100-M-16,Classic,100%,Mini,16,4.48,0.28,0.67
4,CL-100-M-20,Classic,100%,Mini,20,5.60,0.28,0.84


In [11]:
# Define the CSV file path
products_csv_path = '/content/products_complete.csv'

# Save the DataFrame to a CSV file without the index
df_final.to_csv(products_csv_path, index=False)

print(f"products_complete.csv has been created with {len(df_final)} entries at {products_csv_path}.")

products_complete.csv has been created with 525 entries at /content/products_complete.csv.


In [12]:
# from google.colab import files

# # Download the CSV file
# files.download(products_csv_path)

# Customers Data

In [13]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime
from itertools import product
import string
import csv

In [14]:
# Initialize Faker with US locale for consistency
fake = Faker('en_US')

# Seed Faker and random for reproducibility
Faker.seed(0)
random.seed(0)

In [15]:
# Primary and secondary countries
primary_country = 'United States'
secondary_countries = ['Ireland', 'United Kingdom']

# Define Southern California cities and their ZIP code prefixes
southern_california_cities = {
    'Los Angeles': ['90001', '90002', '90003', '90004', '90005', '90006', '90007', '90008', '90009', '90010',
                   '90011', '90012', '90013', '90014', '90015', '90016', '90017', '90018', '90019', '90020'],
    'Long Beach': ['90802', '90803', '90804', '90805', '90806', '90807', '90808', '90810', '90813', '90814'],
    'Pasadena': ['91101', '91103', '91104', '91105', '91106', '91107', '91108', '91109'],
    'Santa Monica': ['90401', '90402', '90403', '90404', '90405', '90406', '90407', '90408'],
    'Irvine': ['92602', '92603', '92604', '92606', '92612', '92614', '92618', '92620', '92623', '92626'],
    'Burbank': ['91501', '91502', '91504', '91505', '91506', '91507', '91508', '91510'],
    'Glendale': ['91201', '91202', '91203', '91204', '91205', '91206', '91207', '91208', '91210'],
    'Anaheim': ['92801', '92802', '92804', '92805', '92806', '92807', '92808', '92809', '92812', '92814'],
    'Torrance': ['90501', '90502', '90503', '90504', '90505', '90506'],
    'Fullerton': ['92831', '92832', '92833', '92835'],
    'Pomona': ['91766', '91767', '91768'],
    'Costa Mesa': ['92626', '92627', '92628', '92629', '92646'],
    'Huntington Beach': ['92605', '92610', '92620', '92647', '92648'],
}

# Define a list of other U.S. cities with their ZIP code prefixes
other_us_cities = {
    'New York City': ['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010'],
    'Chicago': ['60601', '60602', '60603', '60604', '60605'],
    'Houston': ['77001', '77002', '77003', '77004', '77005'],
    'Phoenix': ['85001', '85002', '85003', '85004', '85005'],
    'San Francisco': ['94102', '94103', '94104', '94105', '94107'],
}

In [16]:
# Flatten the list of Southern California cities and ZIP codes for easy selection
la_cities = list(southern_california_cities.keys())
la_zip_codes = [zip_code for zips in southern_california_cities.values() for zip_code in zips]

# Flatten the list of other U.S. cities and ZIP codes
other_us_cities_list = list(other_us_cities.keys())
other_us_zip_codes = [zip_code for zips in other_us_cities.values() for zip_code in zips]

In [17]:
# Define probability distributions
# 90% from Southern California, 7% from other U.S. cities, 3% from foreign countries
location_probabilities = {
    'Southern California': 0.90,
    'Other US': 0.07,
    'Foreign': 0.03
}

# Precompute the cumulative distribution for location selection
cumulative_probs = []
current = 0
for key, prob in location_probabilities.items():
    current += prob
    cumulative_probs.append((current, key))

In [18]:
def select_location():
    """
    Selects a location type based on predefined probabilities.
    """
    rand = random.random()
    for threshold, location_type in cumulative_probs:
        if rand <= threshold:
            return location_type
    return 'Southern California'  # Fallback

def generate_customer_id(existing_ids):
    """
    Generates a unique Customer ID in the format '#####-#####-??'.
    Ensures uniqueness by checking against existing_ids.
    """
    while True:
        customer_id = fake.unique.bothify(text='#####-#####-??').upper()
        if customer_id not in existing_ids:
            existing_ids.add(customer_id)
            return customer_id

def generate_phone_number():
    """
    Generates a phone number in the format '(XXX) XXX-XXXX'.
    """
    area_code = random.randint(200, 999)  # Avoid area codes starting with 0 or 1
    first_part = random.randint(200, 999)
    second_part = random.randint(1000, 9999)
    return f"({area_code}) {first_part}-{second_part}"

In [19]:
def generate_customer(existing_ids, existing_emails):
    """
    Generates a single customer entry with unique Customer ID and Email.
    """
    location_type = select_location()

    if location_type == 'Southern California':
        city = random.choice(la_cities)
        zip_code = random.choice(southern_california_cities[city])
        country = primary_country
    elif location_type == 'Other US':
        city = random.choice(other_us_cities_list)
        zip_code = random.choice(other_us_zip_codes)
        country = primary_country
    else:  # Foreign
        city = fake.city()
        zip_code = fake.postcode()
        country = random.choice(secondary_countries)

    # Generate a unique customer ID
    customer_id = generate_customer_id(existing_ids)

    # Generate customer details
    name = fake.name()

    # Generate a unique email
    while True:
        email = fake.unique.email().lower()
        if email not in existing_emails:
            existing_emails.add(email)
            break

    phone = generate_phone_number()
    address = fake.street_address()

    # Assign subscription status with 70% 'Yes' and 30% 'No'
    subscribed = random.choices(['Yes', 'No'], weights=[0.7, 0.3])[0]

    return {
        'Customer ID': customer_id,
        'Name': name,
        'Email': email,
        'Phone': phone,
        'Address': address,
        'City': city,
        'Country': country,
        'Zip Code': zip_code,
        'Subscribed': subscribed
    }

In [20]:
def generate_customers(num_customers):
    """
    Generates a DataFrame containing customer data.
    """
    customers = []
    existing_ids = set()
    existing_emails = set()

    for _ in range(num_customers):
        customer = generate_customer(existing_ids, existing_emails)
        customers.append(customer)

    return pd.DataFrame(customers)

In [21]:
# Set the number of customers
NUM_CUSTOMERS = 1000

# Generate the customers DataFrame
df_customers = generate_customers(NUM_CUSTOMERS)

In [22]:
# Define the CSV file path
customers_csv_path = '/content/customers_complete.csv'

# Save the DataFrame to a CSV file without the index
df_customers.to_csv(customers_csv_path, index=False)

print(f"customers_complete.csv has been created with {len(df_customers)} unique customer entries at {customers_csv_path}.")

customers_complete.csv has been created with 1000 unique customer entries at /content/customers_complete.csv.


In [23]:
# Read the generated CSV file
df_customers = pd.read_csv(customers_csv_path)

# Display the first 5 rows
print("Sample Customers Dataset:")
display(df_customers.head())

Sample Customers Dataset:


,Customer ID,Name,Email,Phone,Address,City,Country,Zip Code,Subscribed
0,66048-76475-LN,Diane Campos,vanessa89@example.org,(241) 465-9376,15781 Hamilton Ridge Apt. 877,Huntington Beach,United States,92647,Yes
1,84080-16097-VP,Michele Brown,johnponce@example.net,(566) 797-4578,15871 Arnold Squares Apt. 858,Houston,United States,77001,Yes
2,39894-71965-KP,Collin Lopez,thomas12@example.org,(833) 456-9725,86848 Melissa Springs,Huntington Beach,United States,92605,No
3,96947-75159-HF,Gabriela Lee,dennisdave@example.org,(301) 947-2208,56012 Brandon Ports Apt. 013,Pasadena,United States,91106,No
4,99161-51090-ML,Karen Acosta,gilesandrew@example.com,(303) 562-8113,1314 Graham Circle,Anaheim,United States,92812,Yes


In [24]:
def perform_validation_checks(df_customers):
    """
    Performs comprehensive validation checks on the customers DataFrame.
    Returns a dictionary with validation results.
    """
    validation_results = {}

    # Ensure 'Zip Code' is string
    df_customers['Zip Code'] = df_customers['Zip Code'].astype(str)

    # 1. Check for Duplicate Customer IDs
    duplicate_ids = df_customers[df_customers['Customer ID'].duplicated(keep=False)]
    num_duplicate_ids = duplicate_ids.shape[0]
    validation_results['duplicate_customer_ids'] = {
        'count': num_duplicate_ids,
        'details': duplicate_ids if num_duplicate_ids > 0 else None
    }

    # 2. Check for Duplicate Emails
    duplicate_emails = df_customers[df_customers['Email'].duplicated(keep=False)]
    num_duplicate_emails = duplicate_emails.shape[0]
    validation_results['duplicate_emails'] = {
        'count': num_duplicate_emails,
        'details': duplicate_emails if num_duplicate_emails > 0 else None
    }

    # 3. Check for Missing Values
    missing_values = df_customers.isnull().sum()
    validation_results['missing_values'] = {
        'total_missing': missing_values.sum(),
        'details': missing_values if missing_values.sum() > 0 else None
    }

    # 4. Check Phone Number Format
    # Expected format: (XXX) XXX-XXXX
    phone_pattern = r'^\(\d{3}\) \d{3}-\d{4}$'
    phone_valid = df_customers['Phone'].str.match(phone_pattern)
    invalid_phone = ~phone_valid
    num_invalid_phone = invalid_phone.sum()
    validation_results['invalid_phone_numbers'] = {
        'count': num_invalid_phone,
        'details': df_customers[invalid_phone] if num_invalid_phone > 0 else None
    }

    # 5. Check ZIP Code Format
    # Assuming U.S. ZIP codes are 5 digits, foreign ZIP codes can vary
    # Here, we'll check U.S. ZIP codes (if country is 'United States') are 5 digits
    us_zip_valid = df_customers.apply(
        lambda row: row['Zip Code'].isdigit() and len(row['Zip Code']) == 5
        if row['Country'] == primary_country else True, axis=1
    )
    invalid_us_zip = ~us_zip_valid
    num_invalid_us_zip = invalid_us_zip.sum()
    validation_results['invalid_us_zip_codes'] = {
        'count': num_invalid_us_zip,
        'details': df_customers[invalid_us_zip] if num_invalid_us_zip > 0 else None
    }

    # 6. Check City and ZIP Code Consistency for Southern California and Other US
    # For simplicity, we'll verify that ZIP codes match the predefined lists
    def is_zip_matching(row):
        if row['Country'] != primary_country:
            return True  # Foreign entries are not validated
        if row['City'] in southern_california_cities:
            return row['Zip Code'] in southern_california_cities[row['City']]
        elif row['City'] in other_us_cities:
            return row['Zip Code'] in other_us_cities[row['City']]
        else:
            return False  # City not recognized for validation

    zip_consistency = df_customers.apply(is_zip_matching, axis=1)
    inconsistent_zip = ~zip_consistency
    num_inconsistent_zip = inconsistent_zip.sum()
    validation_results['inconsistent_city_zip'] = {
        'count': num_inconsistent_zip,
        'details': df_customers[inconsistent_zip] if num_inconsistent_zip > 0 else None
    }

    # 7. Check Subscription Status Values
    valid_subscriptions = df_customers['Subscribed'].isin(['Yes', 'No'])
    num_invalid_subscriptions = (~valid_subscriptions).sum()
    validation_results['invalid_subscriptions'] = {
        'count': num_invalid_subscriptions,
        'details': df_customers[~valid_subscriptions] if num_invalid_subscriptions > 0 else None
    }

    return validation_results

In [25]:
# Perform validation checks
validation_results = perform_validation_checks(df_customers)

In [26]:
def summarize_validation_results(validation_results):
    """
    Summarizes the validation results and prints a pass/fail summary.
    """
    print("--- Validation Summary ---\n")

    # 1. Duplicate Customer IDs
    dup_ids = validation_results['duplicate_customer_ids']
    print("1. Duplicate Customer IDs:")
    print(f"   Number of duplicate Customer IDs: {dup_ids['count']}")
    print("   Pass" if dup_ids['count'] == 0 else "   Fail", end="\n\n")

    # 2. Duplicate Emails
    dup_emails = validation_results['duplicate_emails']
    print("2. Duplicate Emails:")
    print(f"   Number of duplicate Emails: {dup_emails['count']}")
    print("   Pass" if dup_emails['count'] == 0 else "   Fail", end="\n\n")

    # 3. Missing Values
    missing = validation_results['missing_values']
    print("3. Missing Values:")
    print(f"   Total missing values: {missing['total_missing']}")
    print("   Pass" if missing['total_missing'] == 0 else "   Fail", end="\n\n")

    # 4. Phone Number Format
    invalid_phone = validation_results['invalid_phone_numbers']
    print("4. Phone Number Format:")
    print(f"   Number of invalid Phone Numbers: {invalid_phone['count']}")
    print("   Pass" if invalid_phone['count'] == 0 else "   Fail", end="\n\n")

    # 5. ZIP Code Format
    invalid_zip = validation_results['invalid_us_zip_codes']
    print("5. ZIP Code Format:")
    print(f"   Number of invalid U.S. ZIP Codes: {invalid_zip['count']}")
    print("   Pass" if invalid_zip['count'] == 0 else "   Fail", end="\n\n")

    # 6. City and ZIP Code Consistency
    inconsistent_zip = validation_results['inconsistent_city_zip']
    print("6. City and ZIP Code Consistency:")
    print(f"   Number of inconsistent City and ZIP Code pairs: {inconsistent_zip['count']}")
    print("   Pass" if inconsistent_zip['count'] == 0 else "   Fail", end="\n\n")

    # 7. Subscription Status Values
    invalid_subs = validation_results['invalid_subscriptions']
    print("7. Subscription Status Values:")
    print(f"   Number of invalid Subscription Status entries: {invalid_subs['count']}")
    print("   Pass" if invalid_subs['count'] == 0 else "   Fail", end="\n\n")

    print("--- End of Validation Summary ---\n")

In [27]:
# Summarize validation results
summarize_validation_results(validation_results)

--- Validation Summary ---

1. Duplicate Customer IDs:
   Number of duplicate Customer IDs: 0
   Pass

2. Duplicate Emails:
   Number of duplicate Emails: 0
   Pass

3. Missing Values:
   Total missing values: 0
   Pass

4. Phone Number Format:
   Number of invalid Phone Numbers: 0
   Pass

5. ZIP Code Format:
   Number of invalid U.S. ZIP Codes: 0
   Pass

6. City and ZIP Code Consistency:
   Number of inconsistent City and ZIP Code pairs: 56
   Fail

7. Subscription Status Values:
   Number of invalid Subscription Status entries: 0
   Pass

--- End of Validation Summary ---



In [28]:
# from google.colab import files

# # Download the CSV file
# files.download(customers_csv_path)

# Orders Data

In [29]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
from itertools import product
import string

In [30]:
# Load the products dataset
products_csv_path = '/content/products_complete.csv'
df_products = pd.read_csv(products_csv_path)

# Load the customers dataset
customers_csv_path = '/content/customers_complete.csv'
df_customers = pd.read_csv(customers_csv_path)

# Display the first few rows of each dataset to verify
print("Products Dataset:")
display(df_products.head())

print("\nCustomers Dataset:")
display(df_customers.head())

Products Dataset:


,Product ID,Flavor,Sweetness Level,Pearl Type,Size (oz),Unit Price ($),Price per oz ($),Profit ($)
0,CL-100-C-16,Classic,100%,Classic,16,4.00,0.25,0.60
1,CL-100-C-20,Classic,100%,Classic,20,5.00,0.25,0.75
2,CL-100-C-24,Classic,100%,Classic,24,6.00,0.25,0.90
3,CL-100-M-16,Classic,100%,Mini,16,4.48,0.28,0.67
4,CL-100-M-20,Classic,100%,Mini,20,5.60,0.28,0.84



Customers Dataset:


,Customer ID,Name,Email,Phone,Address,City,Country,Zip Code,Subscribed
0,66048-76475-LN,Diane Campos,vanessa89@example.org,(241) 465-9376,15781 Hamilton Ridge Apt. 877,Huntington Beach,United States,92647,Yes
1,84080-16097-VP,Michele Brown,johnponce@example.net,(566) 797-4578,15871 Arnold Squares Apt. 858,Houston,United States,77001,Yes
2,39894-71965-KP,Collin Lopez,thomas12@example.org,(833) 456-9725,86848 Melissa Springs,Huntington Beach,United States,92605,No
3,96947-75159-HF,Gabriela Lee,dennisdave@example.org,(301) 947-2208,56012 Brandon Ports Apt. 013,Pasadena,United States,91106,No
4,99161-51090-ML,Karen Acosta,gilesandrew@example.com,(303) 562-8113,1314 Graham Circle,Anaheim,United States,92812,Yes


In [31]:
# Initialize Faker
fake = Faker()
Faker.seed(0)
random.seed(0)

# Define the number of orders to generate
NUM_ORDERS = 5000  # Adjust as needed

# Define the date range for orders (e.g., within the last year)
END_DATE = datetime.today()
START_DATE = END_DATE - timedelta(days=365)  # Last 365 days

# Define maximum number of products per order
MAX_PRODUCTS_PER_ORDER = 5  # Adjust as needed

# Define possible quantities per product
MIN_QUANTITY = 1
MAX_QUANTITY = 5

In [32]:
def generate_unique_order_ids(num_ids):
    """
    Generates a set of unique Order IDs in the format 'ABC-12345-678'.
    """
    order_ids = set()
    while len(order_ids) < num_ids:
        prefix = ''.join(random.choices(string.ascii_uppercase, k=3))
        middle = ''.join(random.choices(string.digits, k=5))
        suffix = ''.join(random.choices(string.digits, k=3))
        order_id = f"{prefix}-{middle}-{suffix}"
        order_ids.add(order_id)
    return list(order_ids)

def generate_orders(num_orders, df_customers, df_products, start_date, end_date, max_products_per_order, min_qty, max_qty):
    orders = []

    # Generate unique Order IDs
    unique_order_ids = generate_unique_order_ids(num_orders)

    for order_id in unique_order_ids:
        # Select a random customer
        customer = df_customers.sample(n=1).iloc[0]
        customer_id = customer['Customer ID']

        # Generate a random order date
        order_date = fake.date_between(start_date=start_date, end_date=end_date)

        # Decide how many different products to include in this order
        num_products = random.randint(1, max_products_per_order)

        # Select unique products for this order
        products = df_products.sample(n=num_products, replace=False)

        for _, product in products.iterrows():
            product_id = product['Product ID']

            # Assign a random quantity
            quantity = random.randint(min_qty, max_qty)

            orders.append({
                'Order ID': order_id,
                'Order Date': order_date.strftime('%Y-%m-%d'),
                'Customer ID': customer_id,
                'Product ID': product_id,
                'Quantity': quantity
            })

    return pd.DataFrame(orders)

In [33]:
# Generate the orders DataFrame
df_orders = generate_orders(
    num_orders=NUM_ORDERS,
    df_customers=df_customers,
    df_products=df_products,
    start_date=START_DATE,
    end_date=END_DATE,
    max_products_per_order=MAX_PRODUCTS_PER_ORDER,
    min_qty=MIN_QUANTITY,
    max_qty=MAX_QUANTITY
)

# Display the first few rows of the orders dataset
print("Orders Dataset:")
display(df_orders.head())

Orders Dataset:


,Order ID,Order Date,Customer ID,Product ID,Quantity
0,HFU-97862-556,2024-09-01,39894-71965-KP,BS-075-W-20,2
1,OTM-62962-480,2024-07-31,46872-67267-OW,PF-000-J-16,3
2,OTM-62962-480,2024-07-31,46872-67267-OW,BS-025-P-24,1
3,OTM-62962-480,2024-07-31,46872-67267-OW,MA-050-P-24,1
4,OTM-62962-480,2024-07-31,46872-67267-OW,TR-050-M-20,3


In [34]:
# Validation Checks

# 1. Check if all Customer IDs in orders exist in customers dataset
invalid_customers = ~df_orders['Customer ID'].isin(df_customers['Customer ID'])
num_invalid_customers = invalid_customers.sum()
print(f"Number of orders with invalid Customer IDs: {num_invalid_customers}")

if num_invalid_customers > 0:
    print("Invalid Customer IDs found in orders:")
    display(df_orders[invalid_customers])
else:
    print("All Customer IDs in orders are valid.")

# 2. Check if all Product IDs in orders exist in products dataset
invalid_products = ~df_orders['Product ID'].isin(df_products['Product ID'])
num_invalid_products = invalid_products.sum()
print(f"\nNumber of orders with invalid Product IDs: {num_invalid_products}")

if num_invalid_products > 0:
    print("Invalid Product IDs found in orders:")
    display(df_orders[invalid_products])
else:
    print("All Product IDs in orders are valid.")

# 3. Check for duplicate Product IDs within the same Order ID
duplicate_order_products = df_orders.duplicated(subset=['Order ID', 'Product ID'], keep=False)
num_duplicate_order_products = duplicate_order_products.sum()
print(f"\nNumber of duplicate Product IDs within the same Order ID: {num_duplicate_order_products}")

if num_duplicate_order_products > 0:
    print("Duplicate Product IDs found within the same Order ID:")
    display(df_orders[duplicate_order_products])
else:
    print("No duplicate Product IDs within the same Order ID.")

# 4. Group by Order ID and check if all Order Dates within the group are the same
order_date_consistency = df_orders.groupby('Order ID')['Order Date'].nunique()
inconsistent_orders = order_date_consistency[order_date_consistency > 1].count()
print(f"\nNumber of orders with inconsistent Order Dates: {inconsistent_orders}")

if inconsistent_orders > 0:
    print("Orders with inconsistent Order Dates found:")
    inconsistent_order_ids = order_date_consistency[order_date_consistency > 1].index
    display(df_orders[df_orders['Order ID'].isin(inconsistent_order_ids)])
else:
    print("All orders have consistent Order Dates.")

# 5. Check for any Quantity values less than 1
invalid_quantities = df_orders['Quantity'] < 1
num_invalid_quantities = invalid_quantities.sum()
print(f"\nNumber of orders with invalid Quantity values: {num_invalid_quantities}")

if num_invalid_quantities > 0:
    print("Orders with invalid Quantity values found:")
    display(df_orders[invalid_quantities])
else:
    print("All Quantity values in orders are valid.")

Number of orders with invalid Customer IDs: 0
All Customer IDs in orders are valid.

Number of orders with invalid Product IDs: 0
All Product IDs in orders are valid.

Number of duplicate Product IDs within the same Order ID: 0
No duplicate Product IDs within the same Order ID.

Number of orders with inconsistent Order Dates: 0
All orders have consistent Order Dates.

Number of orders with invalid Quantity values: 0
All Quantity values in orders are valid.


In [35]:
# Validation Summary
print("--- Validation Summary ---\n")

# Customer ID Validation
print("1. Customer ID Validation:")
print(f"Number of invalid Customer IDs: {num_invalid_customers}")
print("Pass" if num_invalid_customers == 0 else "Fail", end="\n\n")

# Product ID Validation
print("2. Product ID Validation:")
print(f"\nNumber of invalid Product IDs: {num_invalid_products}")
print("Pass" if num_invalid_products == 0 else "Fail", end="\n\n")

# Duplicate Product IDs within the same Order ID
print("3. Duplicate Product IDs within the same Order ID:")
print(f"\nNumber of duplicate Product IDs: {num_duplicate_order_products}")
print("Pass" if num_duplicate_order_products == 0 else "Fail", end="\n\n")

# Order Date Consistency
print("4. Order Date Consistency:")
print(f"\nNumber of orders with inconsistent dates: {inconsistent_orders}")
print("Pass" if inconsistent_orders == 0 else "Fail", end="\n\n")

# Quantity Validation
print("5. Quantity Validation:")
print(f"\nNumber of orders with invalid quantities: {num_invalid_quantities}")
print("Pass" if num_invalid_quantities == 0 else "Fail", end="\n\n")

print("--- End of Validation ---")

--- Validation Summary ---

1. Customer ID Validation:
Number of invalid Customer IDs: 0
Pass

2. Product ID Validation:

Number of invalid Product IDs: 0
Pass

3. Duplicate Product IDs within the same Order ID:

Number of duplicate Product IDs: 0
Pass

4. Order Date Consistency:

Number of orders with inconsistent dates: 0
Pass

5. Quantity Validation:

Number of orders with invalid quantities: 0
Pass

--- End of Validation ---


In [36]:
# Define the CSV file path
orders_csv_path = '/content/orders_complete.csv'

# Save the DataFrame to a CSV file without the index
df_orders.to_csv(orders_csv_path, index=False)

print(f"orders_complete.csv has been created with {len(df_orders)} entries at {orders_csv_path}.")

orders_complete.csv has been created with 14960 entries at /content/orders_complete.csv.


In [37]:
# from google.colab import files

# # Download the orders CSV file
# files.download(orders_csv_path)

# Merged Workbook

In [38]:
!pip install -q openpyxl

In [39]:
import pandas as pd
import os
from google.colab import files

In [40]:
# Define the CSV file paths
products_csv = 'products_complete.csv'
customers_csv = 'customers_complete.csv'
orders_csv = 'orders_complete.csv'

# Define the output Excel workbook path
output_excel = 'merged_workbook.xlsx'

In [41]:
# Function to check file existence
def check_files_exist(file_paths):
    missing_files = [file for file in file_paths if not os.path.isfile(file)]
    if missing_files:
        print("The following file(s) are missing:")
        for file in missing_files:
            print(f" - {file}")
        raise FileNotFoundError("Please ensure all CSV files are present in the specified paths.")
    else:
        print("All CSV files found. Proceeding with merging...\n")

In [42]:
# List of CSV files to check
csv_files = [products_csv, customers_csv, orders_csv]
check_files_exist(csv_files)

All CSV files found. Proceeding with merging...



In [43]:
# Read CSV files into DataFrames
try:
    df_products = pd.read_csv(products_csv)
    print(f"Loaded '{products_csv}' successfully with {len(df_products)} records.\n")
except Exception as e:
    print(f"Error loading '{products_csv}': {e}")
    raise

try:
    df_customers = pd.read_csv(customers_csv)
    print(f"Loaded '{customers_csv}' successfully with {len(df_customers)} records.\n")
except Exception as e:
    print(f"Error loading '{customers_csv}': {e}")
    raise

try:
    df_orders = pd.read_csv(orders_csv)
    print(f"Loaded '{orders_csv}' successfully with {len(df_orders)} records.\n")
except Exception as e:
    print(f"Error loading '{orders_csv}': {e}")
    raise

Loaded 'products_complete.csv' successfully with 525 records.

Loaded 'customers_complete.csv' successfully with 1000 records.

Loaded 'orders_complete.csv' successfully with 14960 records.



In [44]:
# Create an Excel writer object using openpyxl as the engine
with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    # Write each DataFrame to its respective sheet
    df_products.to_excel(writer, sheet_name='products', index=False)
    print("Written 'products' DataFrame to 'products' sheet.")

    df_customers.to_excel(writer, sheet_name='customers', index=False)
    print("Written 'customers' DataFrame to 'customers' sheet.")

    df_orders.to_excel(writer, sheet_name='orders', index=False)
    print("Written 'orders' DataFrame to 'orders' sheet.")

    # No need to call writer.save() as the with block handles it
    print(f"\nExcel workbook '{output_excel}' created successfully with all worksheets.\n")

Written 'products' DataFrame to 'products' sheet.
Written 'customers' DataFrame to 'customers' sheet.
Written 'orders' DataFrame to 'orders' sheet.

Excel workbook 'merged_workbook.xlsx' created successfully with all worksheets.



In [45]:
# Function to validate workbook creation and sheet names
def validate_workbook(workbook_path, expected_sheets):
    """
    Validates that the workbook exists and contains the expected sheets.

    Parameters:
        workbook_path (str): Path to the Excel workbook.
        expected_sheets (list): List of expected sheet names.

    Returns:
        bool: True if validation passes, False otherwise.
    """
    if not os.path.isfile(workbook_path):
        print(f"Workbook '{workbook_path}' does not exist.")
        return False

    try:
        # Load the workbook
        excel_file = pd.ExcelFile(workbook_path, engine='openpyxl')
        actual_sheets = excel_file.sheet_names
        print(f"Workbook '{workbook_path}' contains the following sheets: {actual_sheets}\n")

        # Check for expected sheets
        missing_sheets = [sheet for sheet in expected_sheets if sheet not in actual_sheets]
        if missing_sheets:
            print(f"The following expected sheets are missing: {missing_sheets}")
            return False
        else:
            print("All expected sheets are present.")
            return True
    except Exception as e:
        print(f"Error loading workbook '{workbook_path}': {e}")
        return False

In [46]:
# Define expected sheet names
expected_sheets = ['products', 'customers', 'orders']

# Perform workbook validation
is_valid = validate_workbook(output_excel, expected_sheets)

Workbook 'merged_workbook.xlsx' contains the following sheets: ['products', 'customers', 'orders']

All expected sheets are present.


In [47]:
# Function to validate data consistency between CSVs and Excel sheets
def validate_data_consistency(workbook_path, sheet_name, original_df):
    """
    Validates that the data in the Excel sheet matches the original DataFrame.

    Parameters:
        workbook_path (str): Path to the Excel workbook.
        sheet_name (str): Name of the sheet to validate.
        original_df (pd.DataFrame): Original DataFrame for comparison.

    Returns:
        bool: True if validation passes, False otherwise.
    """
    try:
        # Read the sheet from the workbook
        df_sheet = pd.read_excel(workbook_path, sheet_name=sheet_name, engine='openpyxl')

        # Check row count
        if len(df_sheet) != len(original_df):
            print(f"Row count mismatch in sheet '{sheet_name}': Expected {len(original_df)}, Found {len(df_sheet)}")
            return False

        # Check column names
        if list(df_sheet.columns) != list(original_df.columns):
            print(f"Column mismatch in sheet '{sheet_name}':")
            print(f"Expected columns: {list(original_df.columns)}")
            print(f"Found columns: {list(df_sheet.columns)}")
            return False

        # Optional: Check for any discrepancies in data (e.g., using hashes or sample comparisons)
        # Here, we'll perform a simple comparison on a sample of rows
        sample_original = original_df.sample(n=5, random_state=0).reset_index(drop=True)
        sample_sheet = df_sheet.sample(n=5, random_state=0).reset_index(drop=True)
        comparison = sample_original.equals(sample_sheet)
        if not comparison:
            print(f"Data mismatch found in sheet '{sheet_name}' between original CSV and Excel sheet.")
            return False

        print(f"Data consistency validated for sheet '{sheet_name}'.")
        return True
    except Exception as e:
        print(f"Error validating sheet '{sheet_name}': {e}")
        return False

In [48]:
# Perform data consistency validation for each sheet
print("Validating data consistency for each sheet...\n")
products_valid = validate_data_consistency(output_excel, 'products', df_products)
customers_valid = validate_data_consistency(output_excel, 'customers', df_customers)
orders_valid = validate_data_consistency(output_excel, 'orders', df_orders)

Validating data consistency for each sheet...

Data consistency validated for sheet 'products'.
Data consistency validated for sheet 'customers'.
Data consistency validated for sheet 'orders'.


In [49]:
# Function to summarize validation results
def summarize_validation(workbook_valid, products_valid, customers_valid, orders_valid):
    """
    Summarizes the validation results.
    """
    print("--- Validation Summary ---\n")

    # 1. Workbook existence and sheet names
    print("1. Workbook and Sheet Names Validation:")
    print(f"   Workbook exists and contains all expected sheets: {'Pass' if workbook_valid else 'Fail'}\n")

    # 2. Data consistency for 'products' sheet
    print("2. 'products' Sheet Data Consistency:")
    print(f"   {'Pass' if products_valid else 'Fail'}\n")

    # 3. Data consistency for 'customers' sheet
    print("3. 'customers' Sheet Data Consistency:")
    print(f"   {'Pass' if customers_valid else 'Fail'}\n")

    # 4. Data consistency for 'orders' sheet
    print("4. 'orders' Sheet Data Consistency:")
    print(f"   {'Pass' if orders_valid else 'Fail'}\n")

    # Overall validation
    overall_pass = workbook_valid and products_valid and customers_valid and orders_valid
    print(f"Overall Validation Result: {'Pass' if overall_pass else 'Fail'}")
    print("\n--- End of Validation Summary ---\n")

    return overall_pass

In [50]:
# Summarize the validation results
summarize_validation(is_valid, products_valid, customers_valid, orders_valid)

--- Validation Summary ---

1. Workbook and Sheet Names Validation:
   Workbook exists and contains all expected sheets: Pass

2. 'products' Sheet Data Consistency:
   Pass

3. 'customers' Sheet Data Consistency:
   Pass

4. 'orders' Sheet Data Consistency:
   Pass

Overall Validation Result: Pass

--- End of Validation Summary ---



True

In [51]:
# # Download the Excel workbook to local machine
# files.download(output_excel)

In [52]:
# Final verification message
if is_valid and products_valid and customers_valid and orders_valid:
    print("All validation checks passed successfully. The Excel workbook has been merged and is ready for use.")
else:
    print("Some validation checks failed. Please review the messages above to identify and fix the issues.")

All validation checks passed successfully. The Excel workbook has been merged and is ready for use.
